In [1]:
!git clone https://github.com/asash/gSASRec-pytorch.git temp_folder
!mv temp_folder/* temp_folder/.[!.]* .
!rm -rf temp_folder

Cloning into 'temp_folder'...
remote: Enumerating objects: 182, done.
remote: Counting objects: 100% (182/182), done.
remote: Compressing objects: 100% (139/139), done.
remote: Total 182 (delta 91), reused 111 (delta 36), pack-reused 0 (from 0)
Receiving objects: 100% (182/182), 19.23 MiB | 12.99 MiB/s, done.
Resolving deltas: 100% (91/91), done.


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
!pip install datasets -q
!pip install ir_measures -q
!pip install torchinfo -q
!pip install kagglehub -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.9/287.9 kB 5.7 MB/s eta 0:00:00


In [4]:
import sys
import torch
import requests
import numpy as np
import datasets
import ir_measures
from tqdm import tqdm
import torchinfo
import pandas as pd
import numpy as np
import kagglehub
import shutil

In [5]:
from config import GSASRecExperimentConfig
from dataset_utils import *
from eval_utils import evaluate
from gsasrec import GSASRec
from transformer_decoder import MultiHeadAttention, TransformerBlock

In [6]:
path = kagglehub.dataset_download("retailrocket/ecommerce-dataset")
dest_path = "/content/datasets/retailrocket"
shutil.move(path, dest_path)

100%|██████████| 291M/291M [00:07<00:00, 42.6MB/s]

Extracting files...


'/content/datasets/retailrocket'

In [7]:
import csv
from collections import defaultdict
import numpy as np
import json
from pathlib import Path

DATASET_DIR = Path('/content/datasets/retailrocket/events.csv').parent
TRAIN_DIR = DATASET_DIR / "train"
VAL_DIR = DATASET_DIR / "val"
TEST_DIR = DATASET_DIR / "test"
RAW_FILE = DATASET_DIR / "events.csv"

"""
1) формируем для каждого пользователя последовательность позитивов (addtocart) и негативов (view)
2) Удаляем из негативов те itemid, которые у этого же user оказались в позитиве.
3) Отсеиваем товары (itemid), у которых <3 взаимодействий (view+addtocart).
4) Удаляем пользователей, у которых после этого <3 событий.
5) Переиндексируем itemid в диапазон [1..num_unique_items].
6) формируем на train/val/test:
   - ищем последний и предпоследний ПОЛОЖИТЕЛЬНЫЕ (addtocart) ивенты,
   - Остальные ивенты (включая негативные) до предпоследнего позитива => train,
   - Предпоследний позитив => val/output, всё, что до него => val/input,
   - Последний позитив => test/output, всё, что до него => test/input.
   Если у пользователя <2 позитивов, пропускаем такого пользователя.
7) Для train сохраняем раздельные файлы: input_pos.txt, input_neg.txt (только последовательность до val_idx).
   Для val/test – формат ml1m: input.txt, output.txt.
"""

def preprocess_and_split():
    DATASET_DIR.mkdir(exist_ok=True)
    TRAIN_DIR.mkdir(exist_ok=True)
    VAL_DIR.mkdir(exist_ok=True)
    TEST_DIR.mkdir(exist_ok=True)


    user_events = defaultdict(list)

    with open(RAW_FILE, 'r', encoding='utf-8') as f:
        reader = csv.DictReader(f, delimiter=',')
        for row in reader:
            visitor_id = row['visitorid']
            event_type = row['event']
            item_id = row['itemid']
            timestamp = int(row['timestamp'])

            if event_type not in ['view', 'addtocart']:
                continue

            label = 1 if event_type == 'addtocart' else 0
            user_events[visitor_id].append((timestamp, item_id, label))

    # Шаг 1-2
    user_positive_items = defaultdict(set)
    for u, events in user_events.items():
        for (ts, it, lab) in events:
            if lab == 1:
                user_positive_items[u].add(it)

    for u, events in user_events.items():
        filtered = []
        for (ts, it, lab) in events:
            # если это негатив, но item встречался в позитиве => выкидываем
            if lab == 0 and it in user_positive_items[u]:
                continue
            filtered.append((ts, it, lab))
        user_events[u] = filtered

    # Шаг 3
    item_freq = defaultdict(int)
    for u, events in user_events.items():
        for (ts, it, lab) in events:
            item_freq[it] += 1

    valid_items = {it for it, f in item_freq.items() if f >= 3}

    # Шаг 4
    users_to_delete = []
    for u, events in user_events.items():
        filtered = [(ts, it, lab) for (ts, it, lab) in events if it in valid_items]
        if len(filtered) < 3:
            users_to_delete.append(u)
        else:
            user_events[u] = filtered

    for u in users_to_delete:
        del user_events[u]

    # Шаг 5
    all_items = set()
    for u, events in user_events.items():
        for (ts, it, lab) in events:
            all_items.add(it)
    all_items = sorted(all_items)
    new_id = {}
    for i, old in enumerate(all_items, start=1):
        new_id[old] = i

    for u in user_events:
        tmp = []
        for (ts, it, lab) in user_events[u]:
            tmp.append((ts, new_id[it], lab))
        user_events[u] = tmp


    num_sequences = 0
    num_clicks = 0
    num_purchase = 0

    # Шаг 6
    train_pos_sequences = []
    train_neg_sequences = []
    val_input = []
    val_output = []
    test_input = []
    test_output = []

    for u, events in user_events.items():
        events_sorted = sorted(events, key=lambda x: x[0])  # сорт по времени
        # индексы позитивов
        pos_indices = [i for i, e in enumerate(events_sorted) if e[2] == 1]
        if len(pos_indices) < 2:
            # если <2 позитивов, пропускаем
            continue

        # Предпоследний и последний позитив
        val_idx = pos_indices[-2]
        test_idx = pos_indices[-1]

        # train = события до val_idx
        train_evts = events_sorted[:val_idx]
        val_evt = events_sorted[val_idx]   # валидационный позитив
        test_evt = events_sorted[test_idx] # тестовый позитив

        val_input_events = events_sorted[:val_idx]
        # test_input = всё до test_idx
        test_input_events = events_sorted[:test_idx]

        all_user_events = events_sorted  # для подсчёта кликов/покупок
        num_sequences += 1
        for (_, _, lab) in all_user_events:
            if lab == 0:
                num_clicks += 1
            else:
                num_purchase += 1

        # формируем train_pos/train_neg
        train_pos_list = []
        train_neg_list = []
        for (ts, it, lab) in train_evts:
            if lab == 1:
                train_pos_list.append(it)
            else:
                train_neg_list.append(it)

        train_pos_sequences.append(train_pos_list)
        train_neg_sequences.append(train_neg_list)

        # val
        val_input.append([it for (ts, it, lab) in val_input_events])
        val_output.append(val_evt[1])

        # test
        test_input.append([it for (ts, it, lab) in test_input_events])
        test_output.append(test_evt[1])

    TRAIN_DIR.mkdir(exist_ok=True)
    VAL_DIR.mkdir(exist_ok=True)
    TEST_DIR.mkdir(exist_ok=True)

    with open(TRAIN_DIR / "input_pos.txt", "w") as f:
        for seq in train_pos_sequences:
            f.write(" ".join(map(str, seq)) + "\n")

    with open(TRAIN_DIR / "input_neg.txt", "w") as f:
        for seq in train_neg_sequences:
            f.write(" ".join(map(str, seq)) + "\n")

    with open(VAL_DIR / "input.txt", "w") as f:
        for seq in val_input:
            f.write(" ".join(map(str, seq)) + "\n")

    with open(VAL_DIR / "output.txt", "w") as f:
        for it in val_output:
            f.write(str(it) + "\n")

    with open(TEST_DIR / "input.txt", "w") as f:
        for seq in test_input:
            f.write(" ".join(map(str, seq)) + "\n")

    with open(TEST_DIR / "output.txt", "w") as f:
        for it in test_output:
            f.write(str(it) + "\n")

    num_items = len(all_items)
    num_interactions = num_clicks + num_purchase

    stats = {
        "num_users": num_sequences,
        "num_items": num_items,
        "num_interactions": num_interactions,
        "#sequences": num_sequences,
        "#clicks (view)": num_clicks,
        "#purchase (addtocart)": num_purchase
    }
    print(json.dumps(stats, indent=4))

    with open(DATASET_DIR / "dataset_stats.json", "w") as f:
        json.dump(stats, f, indent=4)


if __name__ == "__main__":
    preprocess_and_split()

{
    "num_users": 7359,
    "num_items": 98043,
    "num_interactions": 168393,
    "#sequences": 7359,
    "#clicks (view)": 131814,
    "#purchase (addtocart)": 36579
}


In [ ]:
# config_retailrocket

from config import GSASRecExperimentConfig
from ir_measures import nDCG, R

config = GSASRecExperimentConfig(
    dataset_name='retailrocket',
    sequence_length=200,
    embedding_dim=128,
    num_heads=1,
    max_batches_per_epoch=100,
    num_blocks=2,
    dropout_rate=0.5,
    negs_per_pos=None,        #
    gbce_t = 0.0,
    reuse_item_embeddings=False,
    metrics=[nDCG@10, R@1, R@10],
    val_metric=nDCG@10,
    early_stopping_patience=20,
    max_epochs=10000
)

In [ ]:
# dataset_utils.py

import json
import torch
from torch.utils.data import Dataset, DataLoader

class SequenceWithNegDataset(Dataset):
    """
    Для train: храним два файла:
      1) input_pos.txt: каждая строка — последовательность позитивных itemid
      2) input_neg.txt: соответствующая строка — последовательность негативных itemid

    При этом предполагается, что длины строк совпадают по индексам
    или, как минимум, они соответствуют одному пользователю, но длины могут различаться.

    В __getitem__(idx) возвращаем (pos_list, neg_list).
    """
    def __init__(self, pos_file, neg_file, padding_value, max_length=200):
        with open(pos_file, 'r') as f:
            self.pos_sequences = [list(map(int, line.strip().split())) for line in f]
        with open(neg_file, 'r') as f:
            self.neg_sequences = [list(map(int, line.strip().split())) for line in f]

        # проверяем, что pos и neg идут в том же порядке пользователей (одинаковое количество строк)
        assert len(self.pos_sequences) == len(self.neg_sequences), \
            "pos_file и neg_file должны иметь одинаковое число строк (одинаковое число пользователей)"

        self.max_length = max_length
        self.padding_value = padding_value

    def __len__(self):
        return len(self.pos_sequences)

    def __getitem__(self, idx):
        pos_seq = self.pos_sequences[idx]
        neg_seq = self.neg_sequences[idx]
        # обрежем или дополним до max_length
        if len(pos_seq) > self.max_length:
            pos_seq = pos_seq[-self.max_length:]
        else:
            pos_seq = [self.padding_value]*(self.max_length - len(pos_seq)) + pos_seq

        if len(neg_seq) > self.max_length:
            neg_seq = neg_seq[-self.max_length:]
        else:
            neg_seq = [self.padding_value]*(self.max_length - len(neg_seq)) + neg_seq

        return torch.tensor(pos_seq, dtype=torch.long), torch.tensor(neg_seq, dtype=torch.long)


class SequenceDataset(Dataset):
    def __init__(self, input_file, padding_value, output_file=None, max_length=200):
        with open(input_file, 'r') as f:
            self.inputs = [list(map(int, line.strip().split())) for line in f]

        if output_file:
            with open(output_file, 'r') as f:
                self.outputs = [int(line.strip()) for line in f]
        else:
            self.outputs = None

        self.max_length = max_length
        self.padding_value = padding_value

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        inp = self.inputs[idx]
        rated = set(inp)
        if len(inp) > self.max_length:
            inp = inp[-self.max_length:]
        else:
            inp = [self.padding_value]*(self.max_length - len(inp)) + inp

        inp_tensor = torch.tensor(inp, dtype=torch.long)

        if self.outputs is not None:
            out_tensor = torch.tensor(self.outputs[idx], dtype=torch.long)
            return inp_tensor, rated, out_tensor
        else:
            return (inp_tensor, )


def collate_val_test(batch):
    # для валидации/теста
    input_tensors = []
    rated_list = []
    output_tensors = []

    for x in batch:
        inp, rated, out = x
        input_tensors.append(inp)
        rated_list.append(rated)
        output_tensors.append(out)

    input_tensors = torch.stack(input_tensors, dim=0)
    output_tensors = torch.stack(output_tensors, dim=0)
    return [input_tensors, rated_list, output_tensors]


def not_to_recommend_collate(batch, pad_value):
    """
    Коллатер для train. На входе список кортежей (pos_seq, neg_seq).
    Нужно сформировать общий батч.
    Каждую позицию в pos_seq считаем позитивным примером (label=1),
    в neg_seq — негативным (label=0).
    Возвращаем: (batch_items, batch_labels),
    где batch_items.shape = [B, 2, max_len], batch_labels.shape = [B, 2, max_len].
    Или собираем их подряд. Здесь есть много вариантов;
    используем "склеить по размерности seq_len" и хранить отдельно mask.
    """

    # batch_pos.shape = [B, max_len], batch_neg.shape = [B, max_len]
    # labels_pos = ones, labels_neg = zeros
    pos_list = []
    neg_list = []
    for (pos_seq, neg_seq) in batch:
        pos_list.append(pos_seq)
        neg_list.append(neg_seq)

    pos_tensor = torch.stack(pos_list, dim=0)  # [B, max_len]
    neg_tensor = torch.stack(neg_list, dim=0)  # [B, max_len]

    # тензор меток
    pos_labels = torch.ones_like(pos_tensor)
    neg_labels = torch.zeros_like(neg_tensor)

    return pos_tensor, neg_tensor, pos_labels, neg_labels


def get_padding_value(dataset_name):
    with open(f"datasets/{dataset_name}/dataset_stats.json", 'r') as f:
        stats = json.load(f)
    padding_value = stats['num_items'] + 1
    return padding_value


def get_num_items(dataset_name):
    with open(f"datasets/{dataset_name}/dataset_stats.json", 'r') as f:
        stats = json.load(f)
    return stats['num_items']


def get_train_dataloader(dataset_name, batch_size=32, max_length=200):
    """
    Возвращаем DataLoader, который читает файлы input_pos.txt и input_neg.txt,
    и коллатит их в батчи с явными негативами.
    """
    dataset_dir = f"datasets/{dataset_name}"
    pad_val = get_padding_value(dataset_name)

    pos_file = f"{dataset_dir}/train/input_pos.txt"
    neg_file = f"{dataset_dir}/train/input_neg.txt"

    train_dataset = SequenceWithNegDataset(pos_file, neg_file, padding_value=pad_val, max_length=max_length)
    train_loader = DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=True,
        collate_fn=lambda b: not_to_recommend_collate(b, pad_val)
    )
    return train_loader


def get_val_dataloader(dataset_name, batch_size=32, max_length=200):
    dataset_dir = f"datasets/{dataset_name}"
    pad_val = get_padding_value(dataset_name)
    inp_file = f"{dataset_dir}/val/input.txt"
    out_file = f"{dataset_dir}/val/output.txt"
    val_dataset = SequenceDataset(inp_file, pad_val, out_file, max_length=max_length)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_val_test)
    return val_loader


def get_test_dataloader(dataset_name, batch_size=32, max_length=200):
    dataset_dir = f"datasets/{dataset_name}"
    pad_val = get_padding_value(dataset_name)
    inp_file = f"{dataset_dir}/test/input.txt"
    out_file = f"{dataset_dir}/test/output.txt"
    test_dataset = SequenceDataset(inp_file, pad_val, out_file, max_length=max_length)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_val_test)
    return test_loader

In [ ]:
from argparse import ArgumentParser
import os

import torch
from torchinfo import summary
from tqdm import tqdm

from utils import load_config, build_model, get_device
from dataset_utils import get_train_dataloader, get_val_dataloader, get_num_items
from eval_utils import evaluate

models_dir = "models"
os.makedirs(models_dir, exist_ok=True)

parser = ArgumentParser()
parser.add_argument('--config', type=str, default='config_retailrocket.py')
parser.add_argument('--use_predefined_neg', action='store_true', default=False,
                    help="при установке флага будем использовать заранее определённые негативы.")

if any('jupyter' in arg for arg in sys.argv) or any('ipykernel' in arg for arg in sys.argv):
    args, unknown = parser.parse_known_args([])
else:
    args, unknown = parser.parse_known_args()

config = load_config(args.config)
num_items = get_num_items(config.dataset_name)
device = get_device()
model = build_model(config).to(device)

train_dataloader = get_train_dataloader(
    config.dataset_name,
    batch_size=config.train_batch_size,
    max_length=config.sequence_length
)
val_dataloader = get_val_dataloader(
    config.dataset_name,
    batch_size=config.eval_batch_size,
    max_length=config.sequence_length
)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

batches_per_epoch = min(config.max_batches_per_epoch, len(train_dataloader))

best_metric = float("-inf")
best_model_name = None
step = 0
steps_not_improved = 0

summary(model, (config.train_batch_size, config.sequence_length), batch_dim=None)

def not_to_recommend_loss(logits, labels, mask):
    """
    logits shape: [B, max_len]
    labels shape: [B, max_len] (0 или 1)
    mask   shape: [B, max_len], 1=считаем в лоссе, 0=пропускаем (padding)
    sum( y*log(sigmoid(logit)) + (1-y)*log(1 - sigmoid(logit)) )
    но в варианте "p = softmax(...)?"
    В gSASRec p обычно получается через скалярное произведение с эмбеддингами,
    но можно и sigmoid. Можно делать logits -> sigma,
    затем -log(sigma) или -log(1-sigma).

    реализация BCE:
    BCE(pos) = -log(sigmoid(logit)),
    BCE(neg) = -log(1 - sigmoid(logit)).
    """
    # считаем p = sigmoid(logits)
    # logits[B, max_len]
    preds = torch.sigmoid(logits)
    # labels: 1 => -log(preds), 0 => -log(1 - preds)
    eps = 1e-8
    loss_pos = -labels * torch.log(preds + eps)
    loss_neg = -(1 - labels) * torch.log(1 - preds + eps)
    loss_elm = loss_pos + loss_neg  # [B, max_len]
    loss_elm = loss_elm * mask      # убираем паддинги
    loss_mean = loss_elm.sum() / (mask.sum() + eps)
    return loss_mean

for epoch in range(config.max_epochs):
    model.train()
    batch_iter = iter(train_dataloader)
    pbar = tqdm(range(batches_per_epoch))
    loss_sum = 0.0

    for _ in pbar:
        step += 1
        try:
            pos_seq, neg_seq, pos_labels, neg_labels = next(batch_iter)
        except StopIteration:
            break

        pos_seq = pos_seq.to(device)  # [B, max_len]
        neg_seq = neg_seq.to(device)
        pos_labels = pos_labels.to(device).float()  # [B, max_len], 1
        neg_labels = neg_labels.to(device).float()  # [B, max_len], 0

        # формируем общий вход (concat).
        # несколько вариантов — обучать Transformer на pos_seq и neg_seq раздельно или чередовать.
        # Делвем:
        #   1) делаем forward отдельно для pos_seq, получаем скрытые представления + head на айтем?
        #   2) тоже самое для neg_seq.
        #   3) считаем лосс.
        # Ниже дыа отдельных прохода.

        # Forward для pos_seq
        last_hidden_pos, _ = model(pos_seq)   # shape [B, max_len, emb]
        # SASRec предсказывает "следующий item" по последнему hidden,
        # но если хотим просто "точечно" получить логиты для самих pos_seq,
        # можно сделать скалярное произв.
        # Например, logits_pos[b, t] = dot( last_hidden_pos[b, t], E[item=t] ).
        # Но проще "head" сделать для каждого item:
        # в gSASRec по умолчанию get_predictions() смотрит на последний шаг.
        # Поэтрму, чтобы воспроизвести логику "по всем позициям", нужнр вычислять logits.
        # допустим: logits = W * hidden (linear), а itemID - часть лейбла.
        # Берем:
        # logits_pos[b,t] = < last_hidden_pos[b,t], output_embedding(pos_seq[b,t]) >

        output_embeddings = model.get_output_embeddings().weight  # shape [num_items+2, emb]
        # собираем вектор для каждого pos_seq[b,t]
        # pos_seq[b,t] — это индекс itemID (с учётом паддинга)
        # для удобства:
        bsz, seqlen = pos_seq.shape
        # создаем logits_pos[b, t] = dot( hidden[b,t], emb_of_item( pos_seq[b,t] ) )
        hidden_dim = last_hidden_pos.size(-1)
        emb_pos = output_embeddings[pos_seq.view(-1)]  # shape [B*seqlen, emb]
        hidden_pos = last_hidden_pos.view(-1, hidden_dim)  # same
        logits_pos = torch.sum(hidden_pos * emb_pos, dim=-1)  # [B*seqlen]
        logits_pos = logits_pos.view(bsz, seqlen)            # [B, seqlen]

        # то же самое для neg_seq
        last_hidden_neg, _ = model(neg_seq)
        emb_neg = output_embeddings[neg_seq.view(-1)]
        hidden_neg = last_hidden_neg.view(-1, hidden_dim)
        logits_neg = torch.sum(hidden_neg * emb_neg, dim=-1).view(bsz, seqlen)

        # итоговые logits и labels
        # посчитаем лосс отдельно и сложим.
        # L = L(pos) + L(neg).
        # mask = 1, если item != pad_value, иначе 0.
        pad_val = num_items + 1
        mask_pos = (pos_seq != pad_val).float()
        mask_neg = (neg_seq != pad_val).float()

        loss_pos_val = not_to_recommend_loss(logits_pos, pos_labels, mask_pos)
        loss_neg_val = not_to_recommend_loss(logits_neg, neg_labels, mask_neg)
        loss = loss_pos_val + loss_neg_val

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        loss_sum += loss.item()
        pbar.set_description(f"Epoch {epoch}, step {step}, loss={loss_sum / (step)}")

    # валидация
    evaluation_result = evaluate(model, val_dataloader, config.metrics,
                                 config.recommendation_limit, config.filter_rated, device=device)
    print(f"Epoch {epoch} val_result: {evaluation_result}")

    metric_value = evaluation_result[config.val_metric]
    if metric_value > best_metric:
        best_metric = metric_value
        model_name = f"{models_dir}/gsasrec-{config.dataset_name}-epoch{epoch}-step{step}-val{best_metric:.4f}.pt"
        if best_model_name and os.path.exists(best_model_name):
            os.remove(best_model_name)
        best_model_name = model_name
        torch.save(model.state_dict(), model_name)
        steps_not_improved = 0
        print(f"New best model saved to {model_name}")
    else:
        steps_not_improved += 1
        if steps_not_improved >= config.early_stopping_patience:
            print("Early stopping triggered.")
            break

100%|██████████| 15/15 [00:04<00:00,  3.70it/s]


Epoch 0 val_result: {R@1: 0.0, nDCG@10: 0.00018670875769996213, R@10: 0.00040766408479412964}
New best model saved to models/gsasrec-retailrocket-epoch0-step58-val0.0002.pt


100%|██████████| 15/15 [00:04<00:00,  3.17it/s]


Epoch 1 val_result: {R@1: 0.00027177605652941976, nDCG@10: 0.000398418229275097, R@10: 0.0005435521130588395}
New best model saved to models/gsasrec-retailrocket-epoch1-step116-val0.0004.pt


100%|██████████| 15/15 [00:04<00:00,  3.69it/s]


...

Epoch 199 val_result: {R@1: 0.0, nDCG@10: 4.286790009318233e-05, R@10: 0.00013588802826470988}


Epoch 200, step 11658, loss=0.00035353338636354386: 100%|██████████| 58/58 [00:02<00:00, 23.13it/s]
100%|██████████| 15/15 [00:04<00:00,  3.40it/s]


Epoch 200 val_result: {R@1: 0.0, nDCG@10: 4.529600942156996e-05, R@10: 0.00013588802826470988}


Epoch 201, step 11716, loss=0.0003344473470677773: 100%|██████████| 58/58 [00:02<00:00, 22.67it/s]
100%|██████████| 15/15 [00:03<00:00,  3.85it/s]


Epoch 201 val_result: {R@1: 0.0, nDCG@10: 4.090637255931257e-05, R@10: 0.00013588802826470988}
Early stopping triggered.
